In [4]:
import scipy.stats as st 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [5]:
from patsy import dmatrices
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_val_score

In [6]:
import pymc3 as pm
from pymc3 import Normal, Binomial, sample, Model # Import relevant distributions
from pymc3.math import invlogit
# Use a theano shared variable to be able to exchange the data the model runs on
from theano import shared

# Simulate Data (Logistic Regression)

In [44]:
## TOY DATA LOGISTIC Regression
#> set.seed(666)
np.random.seed(seed=233423)
x1 = st.norm.rvs(size=10000)           # some continuous variables 
x2 = st.norm.rvs(size=10000)  
z = 1 + 2 *x1 +3 *x2        # linear combination with a bias
pr = 1/(1 +np.exp(-z))         # pass through an inv-logit function
y = st.binom.rvs(n=1,p=pr, size=10000) #rbinom(1000,1,pr) # bernoulli response variable
 
X=np.column_stack([x1,x2])
# standardize the features since regularization requires all features to be on same scale
scaler = StandardScaler(copy=True)
# we have created a standardization based on the training data
X_train = scaler.fit(X).transform(X)
y_train = y

#now feed it to glm:
#df = data.frame(y=y,x1=x1,x2=x2)

# Logistic Regression Bayesian Inference approach :
- Pymc3 with Theano

Steps:
    1. Prepare Data
    2. Build Probabilistic Model
    3. Condition Model on Data & Find the local maximum a posteriori point given a model (MAP)
    4. Sample posterior distribution using MAP as starting points for Indepedent Variables(X's)
    5. Generate posterior predictive samples from model given a Samples of posterior distribution.

In [68]:
# 1 Use theano shared variable so that we can make predictions for new values later
log_dose_shared0 = shared(X_train[:, 0])
log_dose_shared = shared(X_train[:, 1])

# Sample size in each group. The sample size has to be a shared variable too
# Each row/observation is a group so n = total in group. 1 if only one per group
n_shared = shared(np.ones(len(X_train), dtype=int))

# Outcomes/Target
deaths = y_train


# 2 Build Probabilistic Model
with Model() as bioassay_model:

    # Priors for unknown model parameters. e.g. Logit-linear model parameters
    alpha = Normal('alpha', 0, sd=100)
    beta = Normal('beta', 0, sd=100)
    beta0 = Normal('beta0', 0, sd=100)

    # Expected value of outcome. e.g. link function outcome. Calculate probabilities of Y/Target
    theta = invlogit(alpha + beta * log_dose_shared + beta0 * log_dose_shared0 )

    # Likelihood (sampling distribution) of observations Data likelihood YTarget
    obs_deaths = Binomial('obs_deaths', n=n_shared, p=theta, observed=deaths)

    
# 3 Finds the local maximum a posteriori point given a model. uses BFGS.
from pymc3 import find_MAP
# Runs fit to data returns parameters/coefficients
map_estimate = find_MAP(model=bioassay_model)
print(map_estimate)


# 4 Now draw samples from the posterior using the given step methods.
with bioassay_model:
    
    # obtain starting values via MAP
    start = find_MAP(model=bioassay_model)
    
    # instantiate sampler
    step = pm.Metropolis()
    
    # posterior of X's
    # draw 1,000 posterior samples of independent variables
    bioassay_trace = sample(1000, step=step, start=start)


# 5 Generate posterior predictive samples from a model given a trace.
from pymc3 import sample_ppc

with bioassay_model:
    deaths_sim = sample_ppc(bioassay_trace, samples=1000)
    
# take only last half  of posterior distr. of X's. other half was burn in.
tr1 = bioassay_trace[500:]
    
#PREDICT
log_dose_to_predict0 = X_train[:1000,0] #np.random.uniform(-0.8,0.7,size=50)
log_dose_to_predict = X_train[:1000,1] #np.random.uniform(-0.8,0.7,size=50)
n_predict = n = np.ones(1000, dtype=int)

# Changing values here will also change values in the model
log_dose_shared0.set_value(log_dose_to_predict0)
log_dose_shared.set_value(log_dose_to_predict)
n_shared.set_value(n_predict)

# Simply running PPC will use the updated values and do prediction
ppc = pm.sample_ppc(tr1, model=bioassay_model, samples=500)

logp = -3,120.2, ||grad|| = 1.7939: 100%|██████████████████████████████████████████████| 14/14 [00:01<00:00,  9.16it/s]


{'alpha': array(0.9053831851570006), 'beta': array(2.9168537609096776), 'beta0': array(1.9854704735996134)}


logp = -3,120.2, ||grad|| = 1.7939: 100%|██████████████████████████████████████████████| 14/14 [00:01<00:00, 12.06it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1809.41it/s]


In [69]:
print( 'Accuracy:',(ppc['obs_deaths']==y[:1000]).mean() )

Accuracy: 0.805806


In [100]:
X_train[:8]

array([[-0.71438705,  1.04372761],
       [ 0.44338059,  2.91463364],
       [ 0.20930136,  1.06472176],
       [ 2.57440271,  0.08986007],
       [ 0.93622719, -1.2161206 ],
       [-0.57650503, -1.95645393],
       [-0.71714114, -0.10367281],
       [-0.82816794,  0.65578596]])

In [66]:
# X0 , X  columns and  theta formula swapped
# [ 2.57440271,  0.08986007], #Target = 1
# [ 0.93622719, -1.2161206 ] #Target = 0
# Changing values here will also change values in the model
log_dose_shared0.set_value(np.array([ 0.93622719]))
log_dose_shared.set_value(np.array([-1.2161206]))
n_shared.set_value(np.array([1]))

# Simply running PPC will use the updated values and do prediction
ppc = pm.sample_ppc(tr1, model=bioassay_model, samples=500)

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2292.71it/s]


In [99]:
logitInv= lambda x: np.exp(x)/(1.0+np.exp(x)) #sigmoid --> returns probability
logitInv(0.9053831851570006 + 2.9168537609096776 * 0.08986007 + 1.9854704735996134 * 2.57440271)

0.99812803417373919

# Random Forest Classifier

In [101]:
clf = RandomForestClassifier(max_depth=100,max_features='sqrt', random_state=0)
clf.fit(X_train, y_train)

print(clf.feature_importances_)

#print(clf.predict([[0, 0, 0, 0]]))
# not such a great result with our naive random forest...
# we take np.abs because scikit learn returns a negative number for mean_absolute_error by default

print(cross_val_score(clf, X_train, y_train, cv=10 ),
np.abs(cross_val_score(clf, X_train, y_train, cv=10 ).mean()) )
#cross_val_score(model, df, y, cv=6)

[ 0.40173059  0.59826941]
[ 0.84115884  0.82117882  0.836       0.829       0.814       0.834       0.821
  0.832       0.82182182  0.82082082] 0.827098030498


In [103]:
print(clf.predict([[2.57440271,  0.08986007]]))
clf.predict_proba([[2.57440271,  0.08986007]])

[1]


array([[ 0.1,  0.9]])

# Logistic Regression

In [105]:
#logistic Regression
logit = LogisticRegression(fit_intercept=True)

# Fit model. Let X_train = matrix of predictors, y_train = matrix of variable.
# NOTE: Do not include a column for the intercept when fitting the model.
resLogit = logit.fit(X_train, y_train)
print(resLogit.coef_)
print(cross_val_score(resLogit, X_train, y_train, cv=10 ),
      np.abs(cross_val_score(resLogit, X_train, y_train, cv=10 ).mean()) )

[[ 1.9741301   2.90014277]]
[ 0.85514486  0.83716284  0.863       0.845       0.865       0.88        0.86
  0.859       0.84784785  0.85985986] 0.857201540002


# Simulate Data (Random Forest)

In [43]:
#Toy Data Random Forest data
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=4,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)
X_train = scaler.fit(X).transform(X)
y_train = y

In [40]:
#logistic Regression
logit = LogisticRegression(fit_intercept=True)

# Fit model. Let X_train = matrix of predictors, y_train = matrix of variable.
# NOTE: Do not include a column for the intercept when fitting the model.
resLogit = logit.fit(X_train, y_train)
print(resLogit.coef_)
np.abs(cross_val_score(resLogit, X_train, y_train, cv=10 ).mean())

[[ 1.9741301   2.90014277]]


0.85720154000153992

In [20]:
print(clf.predict([[-0.71438705,  1.04372761]]))
#X_train[:20]
#np.count_nonzero(y_train == 1)

[1]


In [22]:
clf.predict_proba([[-0.71438705,  1.04372761]])

array([[ 0.1,  0.9]])

In [27]:
print( np.abs(cross_val_score(clf, X_train, y_train, cv=10 ).mean()),
np.abs(cross_val_score(resLogit, X_train, y_train, cv=10 ).mean()) )

0.827098030498 0.857201540002


In [25]:
# Initiate logistic regression object
logit = LogisticRegression(C=1e9,fit_intercept=True)

# Fit model. Let X_train = matrix of predictors, y_train = matrix of variable.
# NOTE: Do not include a column for the intercept when fitting the model.
resLogit = logit.fit(X_train, y_train)
#print(resLogit.intercept_,resLogit.coef_)

# Calculate matrix of predicted class probabilities. 
# Check resLogit.classes_ to make sure that sklearn ordered your classes as expected
predProbs = np.matrix(resLogit.predict_proba(X_train))

# Design matrix -- add column of 1's at the beginning of your X_train matrix
X_design = np.column_stack((np.ones(shape = X_train.shape[0]), X_train))
#np.ones(shape = X_train.shape[0])
#X_design =X_train

# Initiate matrix of 0's, fill diagonal with each predicted observation's variance
V = np.matrix(np.zeros(shape = (X_design.shape[0], X_design.shape[0])))
np.fill_diagonal(V, np.multiply(predProbs[:,0], predProbs[:,1]).A1)

# Covariance matrix
covLogit = np.linalg.inv(X_design.T * V * X_design)
#print("Covariance matrix: ", covLogit)

# Standard errors
print("Standard errors: ", np.sqrt(np.diag(covLogit)) )

# Wald statistic (coefficient / s.e.) ^ 2
logitParams = np.insert(resLogit.coef_, 0, resLogit.intercept_)
print("Coefficients:    ",logitParams)
#print( "Wald statistics: ", (logitParams / np.sqrt(np.diag(covLogit))) ** 2)

Standard errors:  [ 0.03531992  0.04883996  0.06290314]
Coefficients:     [ 0.90538252  1.98547107  2.91685876]


In [55]:
import statsmodels.formula.api as sm
 
model = sm.Logit(y_train, X_design)
 
result =model.fit() #model.fit(method='bfgs')
result.summary()

Optimization terminated successfully.
         Current function value: 0.310366
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Fri, 29 Sep 2017   Pseudo R-squ.:                  0.5410
Time:                        00:11:01   Log-Likelihood:                -3103.7
converged:                       True   LL-Null:                       -6762.0
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.9054      0.035     25.634      0.000         0.836     0.975
x1             1.9855      0.049     40.653      0.000         1.890     2.081
x2             2.9169      0.063     46.371      0.000         2.794     3.040
==============================================================================
"""

In [56]:
print("Standard errors: ", np.sqrt(np.diag(covLogit)) )
logitParams = np.insert(resLogit.coef_, 0, resLogit.intercept_)
print([round(float(c+(1.96*v)),3) for c,v in zip(logitParams,np.sqrt(np.diag(covLogit)))])
print([round(float(x),3) for x in logitParams])
print([round(float(c-(1.96*v)),3) for c,v in zip(logitParams,np.sqrt(np.diag(covLogit)))])

Standard errors:  [ 0.03531992  0.04883996  0.06290314]
[0.975, 2.081, 3.04]
[0.905, 1.985, 2.917]
[0.836, 1.89, 2.794]


In [30]:
[round(float(c+1.96*v),3) for c,v in zip(logitParams,np.sqrt(np.diag(covLogit)))]
[round(float(c-1.96*v),3) for c,v in zip(logitParams,np.sqrt(np.diag(covLogit)))]

[0.489, 0.831, 1.335]

In [31]:
for c,v in zip(logitParams,np.sqrt(np.diag(covLogit))):
    print(v)

0.355785946657
0.420717909496
0.559726623114


In [32]:
1.1868-(1.96*0.356)

0.48904000000000014

In [ ]:
# import random, math

# def k_fold(data, myseed=11109, k=10):
#     # Load data
#     #data = open(myfile).readlines()

#     # Shuffle input
#     random.seed=myseed
#     random.shuffle(data)

#     # Compute partition size given input k
#     len_part=int(math.ceil(len(data)/float(k)))

#     # Create one partition per fold
#     train={}
#     test={}
#     for ii in range(k):
#         test[ii]  = data[ii*len_part:ii*len_part+len_part]
#         train[ii] = [jj for jj in data if jj not in test[ii]]

#     return train, test 

# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.errorbar(x=log_dose_to_predict0[:50], y=np.asarray(ppc['obs_deaths']).mean(axis=0)[:50], yerr=np.asarray(ppc['obs_deaths']).std(axis=0)[:50], linestyle='', marker='o')
# plt.plot(X_train[:50, 1], deaths[:50], 'o')
# plt.xlabel('log_dose',size=15)s
# plt.ylabel('number of rats with tumors',size=15)

In [ ]:
# rf = RandomForestRegressor(n_estimators = 100, max_features='sqrt')
# rf.fit(X, y)
# # feature importances
# # the higher, the more important the feature
# d = {'importance': rf.feature_importances_}
# pd.DataFrame(d, index=X.columns).sort('importance')